#Step 7
Context generation

In [1]:
import copy

import numpy as np
import matplotlib.pyplot as plt

import Code.utils as utils
import Code.environment.settings as settings
from Code.environment.EnvironmentContextual import EnvironmentContextual
from Code.environment.Customer import Customer
from Code.UCBLearner4 import UCBLearner4
from Code.TSLearner4 import TSLearner4
from Code.ContextManager import ContextManager

%matplotlib inline

In [2]:
cust_json = "customer_classes.json"
today_customer = 25
var_cust = 0
p_lambda = 1.0
graph_json = "business_full_graph.json"

arms = settings.prices
environment = EnvironmentContextual(cust_json,today_customer,var_cust,p_lambda, graph_json, arms)
clairvoyant_arms, clairvoyant_reward, exp_c, rew_per_arm = environment.estimate_clairvoyant()
print("\n", clairvoyant_arms, clairvoyant_reward, exp_c, rew_per_arm.values())

loading: [==============================] 99.6%
 {(0, 0): [3, 3, 1, 2, 0], (0, 1): [3, 2, 3, 3, 0], (1, 0): [3, 2, 1, 0, 3], (1, 1): [3, 2, 3, 1, 0]} {(0, 0): 115.31652784000003, (0, 1): 108.521955072, (1, 0): 181.19790420799998, (1, 1): 219.38617164800007} {(0, 0): 4.6126611136000015, (0, 1): 4.34087820288, (1, 0): 7.24791616832, (1, 1): 8.775446865920003} dict_values([{(0, 0, 0, 0, 0): 24.044481610000005, (0, 0, 0, 0, 1): 11.584068460000003, (0, 0, 0, 0, 2): 23.805472120000005, (0, 0, 0, 0, 3): 25.107092800000004, (0, 0, 0, 1, 0): 29.026339925000006, (0, 0, 0, 1, 1): 16.280885175000005, (0, 0, 0, 1, 2): 28.769365850000007, (0, 0, 0, 1, 3): 30.097093250000007, (0, 0, 0, 2, 0): 34.74790528000001, (0, 0, 0, 2, 1): 21.773996780000004, (0, 0, 0, 2, 2): 34.47195556, (0, 0, 0, 2, 3): 35.819263, (0, 0, 0, 3, 0): 30.039863740000005, (0, 0, 0, 3, 1): 17.348973040000004, (0, 0, 0, 3, 2): 29.78111968, (0, 0, 0, 3, 3): 31.102320400000004, (0, 0, 1, 0, 0): 34.124140450000006, (0, 0, 1, 0, 1): 20.9

In [3]:
clairvoyant_arms

{(0, 0): [3, 3, 1, 2, 0],
 (0, 1): [3, 2, 3, 3, 0],
 (1, 0): [3, 2, 1, 0, 3],
 (1, 1): [3, 2, 3, 1, 0]}

In [4]:
n_arms = 4
n_products = 5 

feature1 = 0
feature2 = 0

TIME_HORIZON = 100
NUM_ITERATION = 1

In [5]:
#unknown parameters
alphas = np.array([0.2, 0.2, 0.2, 0.2, 0.2]) #uniform
prods_dist = np.ones((n_products,n_arms))   #all 1

customers = copy.deepcopy(environment.customers)

for customer in customers:
    customer.set_distribution_alpha(alphas)
    customer.set_num_prods(prods_dist)

UCB

In [6]:
rewards_iterations = []
for i in range(NUM_ITERATION):
    ucb4_learner = ContextManager(UCBLearner4, n_arms, n_products, customers, graph_json, arms, environment.customers_distribution)
    print('Iteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 5 == 0:
            ucb4_learner.build_context()
            print(str(ucb4_learner.tree))
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ucb4_learner.select_superarm()
        print(super_arm)
        report = environment.round(super_arm)
        ucb4_learner.update(super_arm.copy(), report)
    rewards = ucb4_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

Iteration # 1
-1<None, None>
loading: [                              ] 0.0%{(0, 0): [3, 3, 3, 3, 3], (0, 1): [3, 3, 3, 3, 3], (1, 0): [3, 3, 3, 3, 3], (1, 1): [3, 3, 3, 3, 3]}


/home/tpoppo/Documenti/GitHub/OLA_project/Code/ReportSimulation.py:79: RuntimeWarning: invalid value encountered in long_scalars
  return self.reward(prices) / num_customers


{(0, 0): [3, 3, 2, 3, 3], (0, 1): [3, 3, 2, 3, 3], (1, 0): [3, 3, 2, 3, 3], (1, 1): [3, 3, 2, 3, 3]}
{(0, 0): [3, 3, 2, 3, 3], (0, 1): [3, 3, 2, 3, 3], (1, 0): [3, 3, 2, 3, 3], (1, 1): [3, 3, 2, 3, 3]}
{(0, 0): [3, 3, 2, 3, 3], (0, 1): [3, 3, 2, 3, 3], (1, 0): [3, 3, 2, 3, 3], (1, 1): [3, 3, 2, 3, 3]}
{(0, 0): [3, 3, 2, 3, 3], (0, 1): [3, 3, 2, 3, 3], (1, 0): [3, 3, 2, 3, 3], (1, 1): [3, 3, 2, 3, 3]}


/home/tpoppo/Documenti/GitHub/OLA_project/Code/UCBLearner4.py:21: RuntimeWarning: invalid value encountered in true_divide
  customer.set_distribution_alpha(self.estimated_alphas / sum(self.estimated_alphas))
Process ForkPoolWorker-6154:
Process ForkPoolWorker-6155:
Process ForkPoolWorker-6157:
Process ForkPoolWorker-6156:
Process ForkPoolWorker-6158:
Process ForkPoolWorker-6159:
Process ForkPoolWorker-6160:
Process ForkPoolWorker-6153:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.

KeyboardInterrupt: 

In [10]:
ucb4_learner.build_context()

/home/tpoppo/Documenti/GitHub/OLA_project/Code/ReportSimulation.py:79: RuntimeWarning: invalid value encountered in long_scalars
  return self.reward(prices) / num_customers
/home/tpoppo/Documenti/GitHub/OLA_project/Code/UCBLearner4.py:21: RuntimeWarning: invalid value encountered in true_divide
  customer.set_distribution_alpha(self.estimated_alphas / sum(self.estimated_alphas))
Process ForkPoolWorker-6285:
Process ForkPoolWorker-6288:
Process ForkPoolWorker-6283:
Process ForkPoolWorker-6286:
Process ForkPoolWorker-6287:
Process ForkPoolWorker-6284:
Process ForkPoolWorker-6282:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _boot

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 5))
mean = np.mean(rewards_iterations, axis=0)
stdev = np.std(rewards_iterations, axis=0)

plt.plot(range(len(mean)), mean)
plt.fill_between(range(len(mean)), (mean-stdev), (mean+stdev), color='b', alpha=.1)

plt.plot([_ for _ in range(len(rewards))], [clairvoyant_reward for _ in range(len(rewards))])
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
mean_regret = np.mean(regret, axis=0)

plt.plot(range(len(mean_regret)), mean_regret)
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
cumulative_regret = np.cumsum(regret, axis=1)
cum_reg_mean = np.mean(cumulative_regret, axis=0)

plt.plot(range(len(cum_reg_mean)), cum_reg_mean)
plt.show()

TS

In [ ]:
rewards_iterations = []
for i in range(NUM_ITERATION):
    ts_learner4 = ContextManager(TSLearner4, n_arms, n_products, customers, graph_json, arms, environment.customers_distribution)
    print('\nIteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 14 == 0:
            ts_learner4.build_context()
            print(str(ts_learner4.tree))

        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ts_learner4.select_superarm()
        report = environment.round(super_arm)
        ts_learner4.update(super_arm.copy(), report)
    rewards = ts_learner4.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

In [ ]:
plt.figure(figsize=(10, 5))
mean = np.mean(rewards_iterations, axis=0)
stdev = np.std(rewards_iterations, axis=0)

plt.plot(range(len(mean)), mean)
plt.fill_between(range(len(mean)), (mean-stdev), (mean+stdev), color='b', alpha=.1)

plt.plot([_ for _ in range(len(rewards))], [clairvoyant_reward for _ in range(len(rewards))])
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
mean_regret = np.mean(regret, axis=0)

plt.plot(range(len(mean_regret)), mean_regret)
plt.show()

In [ ]:
a = np.mean(regret[:,200:], axis=0)
np.shape(a)
print(np.mean(a))

In [ ]:
plt.figure(figsize=(8,5))
regret = clairvoyant_reward - rewards_iterations 
cumulative_regret = np.cumsum(regret, axis=1)
cum_reg_mean = np.mean(cumulative_regret, axis=0)

plt.plot(range(len(cum_reg_mean)), cum_reg_mean)
plt.show()